In [ ]:
from scipy.spatial.distance import cdist
import pandas as pd
import torch
import torch_geometric
from torch_geometric.data import Data, DataLoader


In [ ]:


# CSVファイルを読み込み
data = pd.read_csv('/home/takakin/Konjac/data/data_2018to2019.csv')

# 「農地以外」を除外し、インデックスをリセット
data = data[data['Crop'] != '農地以外'].reset_index(drop=True)

# 作付品目+発病指数をワンホットエンコーディング
CropDis_dummies = pd.get_dummies(data['Crop+Dis'], prefix='作付')
data = pd.concat([data, CropDis_dummies], axis=1)
data

In [ ]:
# 緯度経度を配列に変換
location = data[['lati', 'long']].values

# 全ての圃場間の距離を計算
distances = cdist(location, location, metric='euclidean')

# エッジの最大距離を設定 
max_distance = 0.001

# グラフデータセットを初期化
graph_dataset = []

In [ ]:
# 各圃場のグラフを作成
for i, row1 in data.iterrows():
    node_features = []
    node_feature = row1[CropDis_dummies.columns].values
    node_features.append(node_feature)

    # 翌年の発病程度 (next_Dis) を二値ラベルに変換
    next_disease_level = row1['next_Dis']
    label = 0 if next_disease_level in [0, 1] else 1

    edges = []
    edge_features = []
    for j, row2 in data.iterrows():
        if i != j:
            distance = distances[i, j]
            if distance <= max_distance:
                # 一定距離以内の圃場のノードとエッジを追加
                # ノード追加
                neighbor_feature = row2[CropDis_dummies.columns].values
                node_features.append(neighbor_feature)

                # エッジ追加
                edges.append([i, j])  
                edge_feature = (max_distance - distance) / max_distance
                edge_features.append(edge_feature)

    # データをPyTorch Tensorに変換
    node_features = torch.tensor(node_features, dtype=torch.float)
    label = torch.tensor([label], dtype=torch.float)
    edges = torch.tensor(edges, dtype=torch.long).t().contiguous()  # (2, E) の形状に変換
    edge_features = torch.tensor(edge_features, dtype=torch.float).view(-1, 1)  # (E, 1) の形状に変換

    # Data オブジェクトを作成
    graph_data = Data(x=node_features, edge_index=edges, edge_attr=edge_features, y=label)
    graph_dataset.append(graph_data)

# 作成したグラフの数を確認
num_graphs = len(graph_dataset)
print(f"作成したグラフの数: {num_graphs}")
graph_dataset
